In [4]:
import pandas as pd 
import numpy as np

In [9]:
def User_For_Genre(genero):
    # Abrimos el archivo parquet en un dataframe
    funcion_3 = pd.read_parquet('games_items.parquet')

    # Filtrar el DataFrame por el género dado
    df_genre = funcion_3[funcion_3['genres'] == genero]

    # si el dataframe esta vacio, lanzar una excepcion 
    if df_genre.empty:
        return {
            "detail": [
                {
                    "loc": ["string", 0],
                    "msg": "El género no existe",
                    "type": "value_error"
                }
            ]
        }
    
    # Obtener el usuario con más horas jugadas
    user_most_played_id = df_genre.groupby('user_id')['playtime_forever'].sum().idxmax()

    # Obtener la acumulación de horas jugadas por año de lanzamiento
    hours_per_year = get_hours_per_year(df_genre)

    # Devolver los resultados en el formato especificado
    return {"Usuario con más horas jugadas para el género {}".format(genero) : user_most_played_id, "Horas jugadas": hours_per_year}

def get_hours_per_year(df_genre):
    hours_per_year = df_genre.groupby('release_date')['playtime_forever'].sum().astype(int).reset_index()
    hours_per_year.columns = ['Año', 'Horas']

    # Cambiar el valor del año a 'desconocido' cuando es igual a 0
    hours_per_year['Año'] = hours_per_year['Año'].apply(lambda x: 'desconocido' if x == 0 else x)

    # Crear el diccionario de resultados
    hours_per_year = hours_per_year.to_dict('records')

    return hours_per_year

In [13]:
# llamar a la funcion
print(User_For_Genre('Design &amp; Illustration'))

{'Usuario con más horas jugadas para el género Design &amp; Illustration': 'BakaLunatic', 'Horas jugadas': [{'Año': 2007, 'Horas': 4}, {'Año': 2008, 'Horas': 1}, {'Año': 2012, 'Horas': 356}, {'Año': 2013, 'Horas': 359}, {'Año': 2014, 'Horas': 132}, {'Año': 2015, 'Horas': 651}, {'Año': 2016, 'Horas': 0}, {'Año': 2017, 'Horas': 0}]}


In [14]:
# FUNCIÓN 4 - DESARROLLADOR CON MÁS JUEGOS RECOMENDADOS Y CON COMENTARIOS POSITIVOS
def best_developer_year(year):

    # Abrimos el archivo parquet en un dataframe
    funcion_4 = pd.read_parquet('games_reviews.parquet')

    # Filtrar el dataframe 'games_reviews' por año, recomendaciones y comentarios positivos
    df_filtered = funcion_4[(funcion_4['release_date'] == year) & (funcion_4['recommend'] == True) & (funcion_4['sentiment_analysis'] == 2)]

    # Agrupar por desarrollador y contar las reseñas
    developer_counts = df_filtered['developer'].value_counts()

    # Ordenar los resultados y seleccionar los tres primeros
    top_developers = developer_counts.nlargest(3)

    # Crear el DataFrame de resultados
    result = pd.DataFrame(top_developers).reset_index()
    result.columns = ['Desarrollador', 'Cantidad']

    # Convertir el DataFrame a un diccionario
    dict_best_developer = result.to_dict('records')

    return dict_best_developer

In [15]:
# llamar a la funcion
print(best_developer_year(2017))

[{'Desarrollador': 'Smartly Dressed Games', 'Cantidad': 2850}, {'Desarrollador': 'Freejam', 'Cantidad': 1020}, {'Desarrollador': 'Studio Wildcard,Instinct Games,Efecto Studios,Virtual Basement LLC', 'Cantidad': 778}]


In [16]:
# FUNCIÓN 4 - DESARROLLADOR CON MÁS JUEGOS RECOMENDADOS Y CON COMENTARIOS POSITIVOS
def best_developer_year(year):

    # Abrimos el archivo parquet en un dataframe
    funcion_4 = pd.read_parquet('games_reviews.parquet')


    # Filtrar el dataframe 'games_reviews' por año, recomendaciones y comentarios positivos
    df_filtered = funcion_4[(funcion_4['release_date'] == year) & (funcion_4['recommend'] == True) & (funcion_4['sentiment_analysis'] == 2)]

    # Agrupar por desarrollador y contar las reseñas
    developer_counts = df_filtered['developer'].value_counts()

    # Ordenar los resultados y seleccionar los tres primeros
    top_developers = developer_counts.nlargest(3)

    # Crear la lista de resultados
    list_best_developer = []
    for i, (developer, count) in enumerate(top_developers.items(), start=1):
        list_best_developer.append({f"Puesto {i}": developer})

    return list_best_developer


In [21]:
# llamar a la funcion
print(best_developer_year(1997))

[{'Puesto 1': 'Oddworld Inhabitants'}, {'Puesto 2': 'Humongous Entertainment'}, {'Puesto 3': 'Interplay Inc.'}]


In [18]:
# abrir el archivo games_reviews.parquet
uniondeitems = pd.read_parquet('games_reviews.parquet')

In [19]:
# mostrar fechas unicas 
print(uniondeitems['release_date'].unique())

[1997 1998 2006 2005 2000 2007 1996 1995 1994 2001 2004 2003 2008 1993
 2009 2002 2010 2011 2013 2012    0 2015 2014 1992 2016 1989 1999 2017
 1991 1990]


In [22]:

# FUNCIÓN 5 - ANÁLISIS DE SENTIMIENTO DE RESEÑAS PARA UN DESARROLLADOR
def desarrollador_reviews_analysis(games_reviews, desarrolladora):

    # Abrimos el archivo parquet en un dataframe
    funcion_5 = pd.read_parquet('games_reviews.parquet')

    # Filtrar el dataframe por el desarrollador
    df_filtered = games_reviews[games_reviews['developer'] == desarrolladora]

    # Verificar si el DataFrame filtrado está vacío y dar una excepción
    if df_filtered.empty:
        return {
            "detail": [
                {
                    "loc": ["string", 0],
                    "msg": "No hay datos para el desarrollador {}".format(desarrolladora),
                    "type": "value_error"
                }
            ]
        }

    # Contar el número de reseñas con análisis de sentimiento positivo, neutral y negativo
    sentiment_counts = df_filtered['sentiment_analysis'].value_counts()

    # Crear el diccionario de resultados
    list_reviews_analysis = {desarrolladora: {'Negativo': sentiment_counts.get(0, 0), 'Positivo': sentiment_counts.get(2, 0)}}

    return list_reviews_analysis

In [25]:
# mostrar el resultado de la funcion
print(desarrollador_reviews_analysis(uniondeitems, 'Valve'))

{'Valve': {'Negativo': 1446, 'Positivo': 7943}}


In [ ]:
def desarrollador_reviews_analysis(desarrolladora):
    # Abrimos el archivo parquet en un dataframe
    funcion_5 = pd.read_parquet('games_reviews.parquet')

    # Filtrar el dataframe por el desarrollador
    df_filtered = funcion_5[funcion_5['developer'] == desarrolladora]

    # Verificar si el DataFrame filtrado está vacío y dar una excepción
    if df_filtered.empty:
        return {
            "detail": [
                {
                    "loc": ["string", 0],
                    "msg": "No hay datos para el desarrollador {}".format(desarrolladora),
                    "type": "value_error"
                }
            ]
        }

    # Contar el número de reseñas con análisis de sentimiento positivo y negativo
    sentiment_counts = df_filtered['sentiment_analysis'].value_counts()

    # Crear el diccionario de resultados
    list_reviews_analysis = {desarrolladora: {'Negativo': sentiment_counts.get(0, 0), 'Positivo': sentiment_counts.get(2, 0)}}

    return list_reviews_analysis

In [26]:
ejemplo = pd.read_parquet('games_reviews.parquet')

In [27]:
ejemplo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121562 entries, 0 to 121561
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   genres              121562 non-null  string 
 1   item_name           121562 non-null  object 
 2   release_date        121562 non-null  int32  
 3   price               121562 non-null  float64
 4   item_id             121562 non-null  int32  
 5   developer           121562 non-null  object 
 6   user_id             121562 non-null  string 
 7   user_url            121562 non-null  string 
 8   recommend           121562 non-null  bool   
 9   year                121562 non-null  int32  
 10  sentiment_analysis  121562 non-null  int32  
dtypes: bool(1), float64(1), int32(4), object(2), string(3)
memory usage: 7.5+ MB


In [ ]:
def desarrollador_reviews_analysis(desarrolladora):
    # Abrimos el archivo parquet en un dataframe
    funcion_5 = pd.read_parquet('games_reviews.parquet')

    # Filtrar el dataframe por el desarrollador
    df_filtered = funcion_5[funcion_5['developer'] == desarrolladora]

    # Verificar si el DataFrame filtrado está vacío y dar una excepción
    if df_filtered.empty:
        return {
            "detail": [
                {
                    "loc": ["string", 0],
                    "msg": "No hay datos para el desarrollador {}".format(desarrolladora),
                    "type": "value_error"
                }
            ]
        }

    # Contar el número de reseñas con análisis de sentimiento positivo y negativo
    sentiment_counts = df_filtered['sentiment_analysis'].value_counts()

    # Crear el diccionario de resultados
    list_reviews_analysis = {desarrolladora: [{'Negative': int(sentiment_counts.get(0, 0)), 'Positive': int(sentiment_counts.get(2, 0))}]}

    # Asegurarse de que el resultado es un diccionario
    if not isinstance(list_reviews_analysis, dict):
        list_reviews_analysis = list_reviews_analysis.to_dict()

    return list_reviews_analysis

In [28]:
 # Abrimos el archivo parquet en un dataframe
df_gmes = pd.read_parquet('games_reviews.parquet')

In [30]:
# mostrar lista de desarrolladores unicos
print(df_gmes['developer'].unique())

['Stainless Games Ltd' 'Valve' 'Outerlight Ltd.' ... 'Troika Games'
 'Neversoft' 'Malfador Machinations']


In [31]:
def developer(desarrollador):
    # Abrimos el archivo parquet en un dataframe
    funcion_1=pd.read_parquet("games.parquet")
    # Filtrar el dataframe por el desarrollador dado
    df_dev = funcion_1[funcion_1['developer'] == desarrollador]

    # Si el dataframe está vacío, lanzar una excepción
    if df_dev.empty:
        return {
            "detail": [
                {
                    "loc": ["string", 0],
                    "msg": "El desarrollador no existe",
                    "type": "value_error"
                }
            ]
        }
    
    # Agrupar por año y contar el número total de juegos y el número de juegos gratuitos
    juegos_por_año = df_dev.groupby('release_date').size()
    juegos_gratis_por_año = df_dev[df_dev['price'] == 0.0].groupby('release_date').size()
    
    # Asegurarse de que juegos_gratis_por_año tenga la misma longitud que juegos_por_año
    juegos_gratis_por_año = juegos_gratis_por_año.reindex(juegos_por_año.index, fill_value=0)
    
    # Calcular el porcentaje de juegos gratis y redondearlo al entero más cercano
    porcentaje_gratis = ((juegos_gratis_por_año / juegos_por_año) * 100).round().astype(int)
    
    # Asignar nombres a las series
    juegos_por_año.name = "Cantidad de artículos"
    juegos_gratis_por_año.name = "Contenidos Gratis"
    porcentaje_gratis.name = "Porcentaje Gratis"
    
    # Unir las series en un DataFrame
    tabla = pd.concat([juegos_por_año, juegos_gratis_por_año, porcentaje_gratis], axis=1).reset_index()
    tabla.columns = ['Año', 'Cantidad de artículos', 'Contenidos Gratis', 'Porcentaje Gratis']
    
    # Agregar el signo de porcentaje a 'Porcentaje Gratis'
    tabla['Porcentaje Gratis'] = tabla['Porcentaje Gratis'].apply(lambda x: f"{x}%")
    
    # Convertir el DataFrame a un diccionario
    dict_developer = tabla.to_dict(orient='records')
    
    return dict_developer

In [37]:
# llamar a la funcion
print(developer('Dev-desconocidos'))

[{'Año': 0, 'Cantidad de artículos': 2049, 'Contenidos Gratis': 331, 'Porcentaje Gratis': '16%'}, {'Año': 1970, 'Cantidad de artículos': 2, 'Contenidos Gratis': 0, 'Porcentaje Gratis': '0%'}, {'Año': 1975, 'Cantidad de artículos': 1, 'Contenidos Gratis': 0, 'Porcentaje Gratis': '0%'}, {'Año': 1980, 'Cantidad de artículos': 1, 'Contenidos Gratis': 0, 'Porcentaje Gratis': '0%'}, {'Año': 1981, 'Cantidad de artículos': 3, 'Contenidos Gratis': 1, 'Porcentaje Gratis': '33%'}, {'Año': 1982, 'Cantidad de artículos': 3, 'Contenidos Gratis': 0, 'Porcentaje Gratis': '0%'}, {'Año': 1983, 'Cantidad de artículos': 3, 'Contenidos Gratis': 0, 'Porcentaje Gratis': '0%'}, {'Año': 1984, 'Cantidad de artículos': 2, 'Contenidos Gratis': 0, 'Porcentaje Gratis': '0%'}, {'Año': 1985, 'Cantidad de artículos': 2, 'Contenidos Gratis': 0, 'Porcentaje Gratis': '0%'}, {'Año': 1987, 'Cantidad de artículos': 5, 'Contenidos Gratis': 0, 'Porcentaje Gratis': '0%'}, {'Año': 1988, 'Cantidad de artículos': 2, 'Contenidos G

In [33]:
probar =pd.read_parquet("games.parquet") 

In [36]:
# desarrollador con mas juegos gratuitos
print(probar[probar['price'] == 0.0]['developer'].value_counts().idxmax())

Dev-desconocidos


In [38]:
def developer(desarrollador):
    # Abrimos el archivo parquet en un dataframe
    funcion_1=pd.read_parquet("games.parquet")
    # Filtrar el dataframe por el desarrollador dado
    df_dev = funcion_1[funcion_1['developer'] == desarrollador]

    # Si el dataframe está vacío, lanzar una excepción
    if df_dev.empty:
        return {
            "detail": [
                {
                    "loc": ["string", 0],
                    "msg": "El desarrollador no existe",
                    "type": "value_error"
                }
            ]
        }
    
    # Agrupar por año y contar el número total de juegos y el número de juegos gratuitos
    juegos_por_año = df_dev.groupby('release_date').size()
    juegos_gratis_por_año = df_dev[df_dev['price'] == 0.0].groupby('release_date').size()
    
    # Asegurarse de que juegos_gratis_por_año tenga la misma longitud que juegos_por_año
    juegos_gratis_por_año = juegos_gratis_por_año.reindex(juegos_por_año.index, fill_value=0)
    
    # Calcular el porcentaje de juegos gratis y redondearlo al entero más cercano
    porcentaje_gratis = ((juegos_gratis_por_año / juegos_por_año) * 100).round().astype(int)
    
    # Asignar nombres a las series
    juegos_por_año.name = "Cantidad de artículos"
    juegos_gratis_por_año.name = "Contenidos Gratis"
    porcentaje_gratis.name = "Porcentaje Gratis"
    
    # Unir las series en un DataFrame
    tabla = pd.concat([juegos_por_año, juegos_gratis_por_año, porcentaje_gratis], axis=1).reset_index()
    tabla.columns = ['Año', 'Cantidad de artículos', 'Contenidos Gratis', 'Porcentaje Gratis']
    
    # Reemplazar 0 con 'Desconocido' en la columna 'Año'
    tabla['Año'] = tabla['Año'].replace(0, 'Desconocido')
    
    # Agregar el signo de porcentaje a 'Porcentaje Gratis'
    tabla['Porcentaje Gratis'] = tabla['Porcentaje Gratis'].apply(lambda x: f"{x}%")
    
    # Convertir el DataFrame a un diccionario
    dict_developer = tabla.to_dict(orient='records')
    
    return dict_developer

In [39]:
print(developer('Dev-desconocidos'))

[{'Año': 'Desconocido', 'Cantidad de artículos': 2049, 'Contenidos Gratis': 331, 'Porcentaje Gratis': '16%'}, {'Año': 1970, 'Cantidad de artículos': 2, 'Contenidos Gratis': 0, 'Porcentaje Gratis': '0%'}, {'Año': 1975, 'Cantidad de artículos': 1, 'Contenidos Gratis': 0, 'Porcentaje Gratis': '0%'}, {'Año': 1980, 'Cantidad de artículos': 1, 'Contenidos Gratis': 0, 'Porcentaje Gratis': '0%'}, {'Año': 1981, 'Cantidad de artículos': 3, 'Contenidos Gratis': 1, 'Porcentaje Gratis': '33%'}, {'Año': 1982, 'Cantidad de artículos': 3, 'Contenidos Gratis': 0, 'Porcentaje Gratis': '0%'}, {'Año': 1983, 'Cantidad de artículos': 3, 'Contenidos Gratis': 0, 'Porcentaje Gratis': '0%'}, {'Año': 1984, 'Cantidad de artículos': 2, 'Contenidos Gratis': 0, 'Porcentaje Gratis': '0%'}, {'Año': 1985, 'Cantidad de artículos': 2, 'Contenidos Gratis': 0, 'Porcentaje Gratis': '0%'}, {'Año': 1987, 'Cantidad de artículos': 5, 'Contenidos Gratis': 0, 'Porcentaje Gratis': '0%'}, {'Año': 1988, 'Cantidad de artículos': 2, '

In [40]:
def user_data(user_id):
    # Abrimos el archivo parquet en un dataframe
    funcion_2 = pd.read_parquet('games_reviews.parquet')

    # Filtrar el dataframe 'games_reviews' por el 'User_id' dado
    df_filtered = funcion_2[funcion_2['user_id'] == user_id]

    # Verificar si el dataframe filtrado está vacío
    if df_filtered.empty:
        return {
            "detail": [
                {
                    "loc": ["string", 0],
                    "msg": f"El usuario {user_id} no existe.",
                    "type": "value_error"
                }
            ]
        }

    # Calcular la cantidad de dinero gastado y redondearlo a dos decimales
    total_spent = round(df_filtered['price'].sum(), 2)

    # Calcular el porcentaje de recomendación
    total_reviews = df_filtered['recommend'].count()
    recommended_reviews = df_filtered[df_filtered['recommend'] == True]['recommend'].count()
    recommendation_percentage = round((recommended_reviews / total_reviews) * 100) if total_reviews > 0 else 0

    # Calcular la cantidad de items
    total_items = df_filtered['item_name'].nunique()

    # Crear el diccionario de resultados
    dict_userdata = {
        "Usuario": user_id,
        "Dinero gastado": f"{total_spent} USD",
        "% de recomendación": f"{recommendation_percentage}%",
        "cantidad de artículos": total_items
    }

    return dict_userdata

In [42]:
# llamar a la funcion
print(user_data("shaman3soul3"))

{'Usuario': 'shaman3soul3', 'Dinero gastado': '9.99 USD', '% de recomendación': '100%', 'cantidad de artículos': 1}


### sistema de recomendación item-item

In [43]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
import pandas as pd
from scipy.sparse import csr_matrix
import pickle

In [44]:
with open ("item_similarity.pkl", "rb") as archivo: 
    modelo_pickle = pickle.load(archivo)

In [ ]:
def recomendacion_juego(item_id):

    # Abrir el archivo parquet en un DataFrame
    data = pd.read_parquet('item_item.parquet')

    with open('item_similarity.pkl', 'rb') as f:
            item_similarity = pickle.load(f)

    # Convertir la columna 'recommend' a valores numéricos
    data['recommend'] = data['recommend'].apply(lambda x: 1 if x else 0)

    # Crear la matriz de utilidad
    matrix = data.pivot_table(index='item_id', values=['recommend', 'sentiment_analysis'])

    # Rellenar los valores faltantes con 0
    matrix.fillna(0, inplace=True)

    # Calcular la similitud del coseno
    item_similarity = cosine_similarity(matrix)

    # Convertir los resultados en un DataFrame
    item_similarity_df = pd.DataFrame(item_similarity, index=matrix.index, columns=matrix.index)
    
    # Obtener los juegos similares al juego dado
    similar_games = item_similarity_df[item_id].sort_values(ascending=False)

    # Obtener los 5 juegos más similares
    similar_games_ids = similar_games.head(6).index.tolist()[1:]  # Excluimos el primer juego porque es el mismo juego

    # Crear un DataFrame con los IDs y los nombres de los juegos
    recommended_games = pd.DataFrame(similar_games_ids, columns=['item_id'])
    recommended_games['item_name'] = recommended_games['item_id'].apply(lambda id: data[data['item_id'] == id]['item_name'].iloc[0])

    return recommended_games

In [45]:
def recomendacion_juego(item_id):

    # Abrir el archivo parquet en un DataFrame
    data = pd.read_parquet('item_item.parquet')

    with open('item_similarity.pkl', 'rb') as f:
            item_similarity = pickle.load(f)

    # Convertir la columna 'recommend' a valores numéricos
    data['recommend'] = data['recommend'].apply(lambda x: 1 if x else 0)

    # Crear la matriz de utilidad
    matrix = data.pivot_table(index='item_id', values=['recommend', 'sentiment_analysis'])

    # Rellenar los valores faltantes con 0
    matrix.fillna(0, inplace=True)

    # Calcular la similitud del coseno
    item_similarity = cosine_similarity(matrix)

    # Convertir los resultados en un DataFrame
    item_similarity_df = pd.DataFrame(item_similarity, index=matrix.index, columns=matrix.index)
    
    # Obtener los juegos similares al juego dado
    similar_games = item_similarity_df[item_id].sort_values(ascending=False)

    # Obtener los 5 juegos más similares
    similar_games_ids = similar_games.head(6).index.tolist()[1:]  # Excluimos el primer juego porque es el mismo juego

    # Crear un diccionario con los IDs y los nombres de los juegos
    recommended_games = {game_id: data[data['item_id'] == game_id]['item_name'].iloc[0] for game_id in similar_games_ids}

    return recommended_games

In [48]:
# Prueba la función con un item_id
recomendacion_juego(10)

{4560: 'Company of Heroes - Legacy Edition',
 222880: 'Insurgency',
 206500: 'AirMech Strike',
 6060: 'Star Wars: Battlefront 2 (Classic, 2005)',
 22380: 'Fallout: New Vegas'}

In [ ]:
def recomendacion_juego(item_id):

    # Abrir el archivo parquet en un DataFrame
    data = pd.read_parquet('item_item.parquet')

    # Si el dataframe está vacío, lanzar una excepción
    if data.empty:
        raise ValueError("El DataFrame está vacío")

    with open('item_similarity.pkl', 'rb') as f:
            item_similarity = pickle.load(f)

    # Convertir la columna 'recommend' a valores numéricos
    data['recommend'] = data['recommend'].apply(lambda x: 1 if x else 0)

    # Crear la matriz de utilidad
    matrix = data.pivot_table(index='item_id', values=['recommend', 'sentiment_analysis'])

    # Rellenar los valores faltantes con 0
    matrix.fillna(0, inplace=True)

    # Calcular la similitud del coseno
    item_similarity = cosine_similarity(matrix)

    # Convertir los resultados en un DataFrame
    item_similarity_df = pd.DataFrame(item_similarity, index=matrix.index, columns=matrix.index)
    
    # Obtener los juegos similares al juego dado
    similar_games = item_similarity_df[item_id].sort_values(ascending=False)

    # Obtener los 5 juegos más similares
    similar_games_ids = similar_games.head(6).index.tolist()[1:]  # Excluimos el primer juego porque es el mismo juego

    # Crear un diccionario con los IDs y los nombres de los juegos
    recommended_games = {game_id: data[data['item_id'] == game_id]['item_name'].iloc[0] for game_id in similar_games_ids}

    return recommended_games

In [ ]:
# Prueba la función con un item_id
recomendacion_juego(1)

In [51]:
def recomendacion_juego(item_id):

    # Abrir el archivo parquet en un DataFrame
    data = pd.read_parquet('item_item.parquet')

    # Si el dataframe está vacío, lanzar una excepción
    if data.empty:
        return {
            "detail": [
                {
                    "loc": ["string", 0],
                    "msg": "El DataFrame está vacío",
                    "type": "value_error"
                }
            ]
        }

    with open('item_similarity.pkl', 'rb') as f:
            item_similarity = pickle.load(f)

    # Convertir la columna 'recommend' a valores numéricos
    data['recommend'] = data['recommend'].apply(lambda x: 1 if x else 0)

    # Crear la matriz de utilidad
    matrix = data.pivot_table(index='item_id', values=['recommend', 'sentiment_analysis'])

    # Rellenar los valores faltantes con 0
    matrix.fillna(0, inplace=True)

    # Calcular la similitud del coseno
    item_similarity = cosine_similarity(matrix)

    # Convertir los resultados en un DataFrame
    item_similarity_df = pd.DataFrame(item_similarity, index=matrix.index, columns=matrix.index)
    
    # Obtener los juegos similares al juego dado
    similar_games = item_similarity_df[item_id].sort_values(ascending=False)

    # Obtener los 5 juegos más similares
    similar_games_ids = similar_games.head(6).index.tolist()[1:]  # Excluimos el primer juego porque es el mismo juego

    # Crear un diccionario con los IDs y los nombres de los juegos
    recommended_games = {game_id: data[data['item_id'] == game_id]['item_name'].iloc[0] for game_id in similar_games_ids}

    return recommended_games

In [ ]:
# Prueba la función con un item_id
recomendacion_juego(1)

In [53]:
def recomendacion_juego(item_id):

    # Abrir el archivo parquet en un DataFrame
    data = pd.read_parquet('item_item.parquet')

    # Si el dataframe está vacío, devolver un mensaje de error
    if data.empty:
        return {
            "detail": [
                {
                    "loc": ["string", 0],
                    "msg": "El DataFrame está vacío",
                    "type": "value_error"
                }
            ]
        }

    # Si el item_id no existe en el DataFrame, devolver un mensaje de error
    if item_id not in data['item_id'].values:
        return {
            "detail": [
                {
                    "loc": ["string", 0],
                    "msg": "El item_id proporcionado no existe",
                    "type": "value_error"
                }
            ]
        }

    with open('item_similarity.pkl', 'rb') as f:
            item_similarity = pickle.load(f)

    # Convertir la columna 'recommend' a valores numéricos
    data['recommend'] = data['recommend'].apply(lambda x: 1 if x else 0)

    # Crear la matriz de utilidad
    matrix = data.pivot_table(index='item_id', values=['recommend', 'sentiment_analysis'])

    # Rellenar los valores faltantes con 0
    matrix.fillna(0, inplace=True)

    # Calcular la similitud del coseno
    item_similarity = cosine_similarity(matrix)

    # Convertir los resultados en un DataFrame
    item_similarity_df = pd.DataFrame(item_similarity, index=matrix.index, columns=matrix.index)
    
    # Obtener los juegos similares al juego dado
    similar_games = item_similarity_df[item_id].sort_values(ascending=False)

    # Obtener los 5 juegos más similares
    similar_games_ids = similar_games.head(6).index.tolist()[1:]  # Excluimos el primer juego porque es el mismo juego

    # Crear un diccionario con los IDs y los nombres de los juegos
    recommended_games = {game_id: data[data['item_id'] == game_id]['item_name'].iloc[0] for game_id in similar_games_ids}

    return recommended_games

In [56]:
# Prueba la función con un item_id
recomendacion_juego(70)

{265630: 'Fistful of Frags',
 42910: 'Magicka',
 10180: 'Call of Duty®: Modern Warfare® 2',
 212800: 'Super Crate Box',
 550: 'Left 4 Dead 2'}

In [61]:
def recomendacion_juego(item_id):

    # Abrir el archivo parquet en un DataFrame
    data = pd.read_parquet('item_item.parquet')

    # Si el dataframe está vacío, devolver un mensaje de error
    if data.empty:
        return {
            "detail": [
                {
                    "loc": ["string", 0],
                    "msg": "El DataFrame está vacío",
                    "type": "value_error"
                }
            ]
        }

    # Si el item_id no existe en el DataFrame, devolver un mensaje de error
    if item_id not in data['item_id'].values:
        return {
            "detail": [
                {
                    "loc": ["string", 0],
                    "msg": "El item_id proporcionado no existe",
                    "type": "value_error"
                }
            ]
        }

    with open('item_similarity.pkl', 'rb') as f:
            item_similarity = pickle.load(f)

    # Convertir la columna 'recommend' a valores numéricos
    data['recommend'] = data['recommend'].apply(lambda x: 1 if x else 0)

    # Crear la matriz de utilidad
    matrix = data.pivot_table(index='item_id', values=['recommend', 'sentiment_analysis'])

    # Rellenar los valores faltantes con 0
    matrix.fillna(0, inplace=True)

    # Calcular la similitud del coseno
    item_similarity = cosine_similarity(matrix)

    # Convertir los resultados en un DataFrame
    item_similarity_df = pd.DataFrame(item_similarity, index=matrix.index, columns=matrix.index)
    
    # Obtener los juegos similares al juego dado
    similar_games = item_similarity_df[item_id].sort_values(ascending=False)

    # Obtener los 5 juegos más similares
    similar_games_ids = similar_games.head(6).index.tolist()[1:]  # Excluimos el primer juego porque es el mismo juego

    # Crear un diccionario con los IDs y los nombres de los juegos
    recommended_games = {game_id: data[data['item_id'] == game_id]['item_name'].iloc[0] for game_id in similar_games_ids}

    return { "Juegos similares al id ingresado": recommended_games}

In [63]:
# Prueba la función con un item_id
recomendacion_juego(10)

{'Juegos similares al id ingresado': {4560: 'Company of Heroes - Legacy Edition',
  222880: 'Insurgency',
  206500: 'AirMech Strike',
  6060: 'Star Wars: Battlefront 2 (Classic, 2005)',
  22380: 'Fallout: New Vegas'}}

In [64]:
 # Abrir el archivo parquet en un DataFrame
data_prueba = pd.read_parquet('item_item.parquet')

In [65]:
data_prueba.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121562 entries, 0 to 121561
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   item_id             121562 non-null  int32 
 1   recommend           121562 non-null  int64 
 2   sentiment_analysis  121562 non-null  int32 
 3   item_name           121562 non-null  object
dtypes: int32(2), int64(1), object(1)
memory usage: 2.8+ MB


In [66]:
# SISTEMA DE RECOMENDACIÓN - ITEM-ITEM
def recomendacion_juego(item_id):

    # Abrir el archivo parquet en un DataFrame
    data = pd.read_parquet('item_item.parquet')

    # Si el dataframe está vacío, devolver un mensaje de error
    if data.empty:
        return {
            "detail": [
                {
                    "loc": ["string", 0],
                    "msg": "El DataFrame está vacío",
                    "type": "value_error"
                }
            ]
        }

    # Si el item_id no existe en el DataFrame, devolver un mensaje de error
    if item_id not in data['item_id'].values:
        return {
            "detail": [
                {
                    "loc": ["string", 0],
                    "msg": "El item_id proporcionado no existe",
                    "type": "value_error"
                }
            ]
        }

    # with open('item_similarity.pkl', 'rb') as f:
    #         item_similarity = pickle.load(f)

    # Convertir la columna 'recommend' a valores numéricos
    data['recommend'] = data['recommend'].apply(lambda x: 1 if x else 0)

    # Crear la matriz de utilidad
    matrix = data.pivot_table(index='item_id', values=['recommend', 'sentiment_analysis'])

    # Rellenar los valores faltantes con 0
    matrix.fillna(0, inplace=True)

    # Calcular la similitud del coseno
    item_similarity = cosine_similarity(matrix)

    # Convertir los resultados en un DataFrame
    item_similarity_df = pd.DataFrame(item_similarity, index=matrix.index, columns=matrix.index)
    
    # Obtener los juegos similares al juego dado
    similar_games = item_similarity_df[item_id].sort_values(ascending=False)

    # Obtener los 5 juegos más similares
    similar_games_ids = similar_games.head(6).index.tolist()[1:]  # Excluimos el primer juego porque es el mismo juego

    # Crear un diccionario con los IDs y los nombres de los juegos
    recommended_games = {game_id: data[data['item_id'] == game_id]['item_name'].iloc[0] for game_id in similar_games_ids}

    return { "Juegos similares al id ingresado": recommended_games}

In [67]:
# Prueba la función con un item_id
recomendacion_juego(10)

{'Juegos similares al id ingresado': {4560: 'Company of Heroes - Legacy Edition',
  222880: 'Insurgency',
  206500: 'AirMech Strike',
  6060: 'Star Wars: Battlefront 2 (Classic, 2005)',
  22380: 'Fallout: New Vegas'}}

Elegido

In [ ]:
def recomendacion_juego(item_id):

    # Abrir el archivo parquet en un DataFrame
    data = pd.read_parquet('item_item.parquet')

    # Si el dataframe está vacío, devolver un mensaje de error
    if data.empty:
        return {
            "detail": [
                {
                    "loc": ["string", 0],
                    "msg": "El DataFrame está vacío",
                    "type": "value_error"
                }
            ]
        }

    # Si el item_id no existe en el DataFrame, devolver un mensaje de error
    if item_id not in data['item_id'].values:
        return {
            "detail": [
                {
                    "loc": ["string", 0],
                    "msg": "El item_id proporcionado no existe",
                    "type": "value_error"
                }
            ]
        }

    # with open('item_similarity.pkl', 'rb') as f:
    #         item_similarity = pickle.load(f)

    # Convertir la columna 'recommend' a valores numéricos
    data['recommend'] = data['recommend'].apply(lambda x: 1 if x else 0)

    # Crear la matriz de utilidad
    matrix = data.pivot_table(index='item_id', values=['recommend', 'sentiment_analysis'])

    # Rellenar los valores faltantes con 0
    matrix.fillna(0, inplace=True)

    # Calcular la similitud del coseno
    item_similarity = cosine_similarity(matrix)

    # Convertir los resultados en un DataFrame
    item_similarity_df = pd.DataFrame(item_similarity, index=matrix.index, columns=matrix.index)
    
    # Obtener los juegos similares al juego dado
    similar_games = item_similarity_df[item_id].sort_values(ascending=False)

    # Obtener los 5 juegos más similares
    similar_games_ids = similar_games.head(6).index.tolist()[1:]  # Excluimos el primer juego porque es el mismo juego

    # Crear un diccionario con los IDs y los nombres de los juegos
    recommended_games = {game_id: data[data['item_id'] == game_id]['item_name'].iloc[0] for game_id in similar_games_ids}

    return { "Juegos similares al id ingresado": recommended_games}

In [68]:
data_prueba.head(5)

,item_id,recommend,sentiment_analysis,item_name
0,282010,1,1,Carmageddon Max Pack
1,282010,1,1,Carmageddon Max Pack
2,282010,1,1,Carmageddon Max Pack
3,70,1,0,Half-Life
4,70,1,0,Half-Life


In [72]:
# abrir un archivo parquet
data_prueba = pd.read_parquet('item_item.parquet')

In [73]:
data_prueba.head(5)

,item_id,recommend,sentiment_analysis,item_name
0,282010,1,1,Carmageddon Max Pack
1,282010,1,1,Carmageddon Max Pack
2,282010,1,1,Carmageddon Max Pack
3,70,1,0,Half-Life
4,70,1,0,Half-Life


In [70]:
  # Convertir la columna 'recommend' a valores numéricos
data_prueba['recommend'] = data_prueba['recommend'].apply(lambda x: 1 if x else 0)

In [71]:
data_prueba.head(5)

,item_id,recommend,sentiment_analysis,item_name
0,282010,1,1,Carmageddon Max Pack
1,282010,1,1,Carmageddon Max Pack
2,282010,1,1,Carmageddon Max Pack
3,70,1,0,Half-Life
4,70,1,0,Half-Life


In [74]:
# valore unicos de la columna sentiment_analysis
data_prueba['sentiment_analysis'].unique()

array([1, 0, 2])

In [ ]:
# SISTEMA DE RECOMENDACIÓN - ITEM-ITEM
def recomendacion_juego(item_id):
    # Abrir el archivo parquet en un DataFrame
    data = pd.read_parquet('item_item.parquet')

    # Si el dataframe está vacío, devolver un mensaje de error
    if data.empty:
        return {
            "detail": [
                {
                    "loc": ["string", 0],
                    "msg": "El DataFrame está vacío",
                    "type": "value_error"
                }
            ]
        }

    # Si el item_id no existe en el DataFrame, devolver un mensaje de error
    if item_id not in data['item_id'].values:
        return {
            "detail": [
                {
                    "loc": ["string", 0],
                    "msg": "El item_id proporcionado no existe",
                    "type": "value_error"
                }
            ]
        }
    # Esta excepción carga la matriz de similitud de items_similarity que contiene la operación de calcular la similitud del coseno 
    try:
        with open('item_similarity.pkl', 'rb') as f:
            item_similarity_df = pickle.load(f)
    except Exception as e:
        print(f"Ocurrió un error al cargar el archivo: {e}")
        
        # Ajuste manual de valores en la columna 'sentiment_analysis'
        data['sentiment_analysis'] = data['sentiment_analysis'].apply(lambda x: 1 if x == 2 else 0)


        # Convertir la columna 'recommend' a valores numéricos
        data['recommend'] = data['recommend'].apply(lambda x: 1 if x else 0)

        # Crear la matriz de utilidad
        matrix = data.pivot_table(index='item_id', values=['recommend', 'sentiment_analysis'])

        # Rellenar los valores faltantes con 0
        matrix.fillna(0, inplace=True)

        # Calcular la similitud del coseno
        item_similarity = cosine_similarity(matrix)

        # Convertir los resultados en un DataFrame
        item_similarity_df = pd.DataFrame(item_similarity, index=matrix.index, columns=matrix.index)
    
    # Obtener los juegos similares al juego dado
    similar_games = item_similarity_df[item_id].sort_values(ascending=False)

    # Obtener los 5 juegos más similares
    similar_games_ids = similar_games.head(6).index.tolist()[1:]  # Excluimos el primer juego porque es el mismo juego

    # Crear un diccionario con los IDs y los nombres de los juegos
    recommended_games = {game_id: data[data['item_id'] == game_id]['item_name'].iloc[0] for game_id in similar_games_ids}

    return { "Juegos similares al id ingresado": recommended_games}


In [76]:
data = pd.read_parquet('item_item.parquet')
# Ajuste manual de valores en la columna 'sentiment_analysis'
data['sentiment_analysis'] = data['sentiment_analysis'].apply(lambda x: 1 if x == 2 else 0)

 # Convertir la columna 'recommend' a valores numéricos
data['recommend'] = data['recommend'].apply(lambda x: 1 if x else 0)

# Crear la matriz de utilidad
matrix = data.pivot_table(index='item_id', values=['recommend', 'sentiment_analysis'])

# Rellenar los valores faltantes con 0
matrix.fillna(0, inplace=True)

# Calcular la similitud del coseno
item_similarity = cosine_similarity(matrix)

# Convertir los resultados en un DataFrame
item_similarity_df = pd.DataFrame(item_similarity, index=matrix.index, columns=matrix.index)

In [77]:
item_similarity_df.head(5)

item_id,10,20,30,40,50,60,70,80,130,220,...,512540,512630,514520,516040,520550,521340,521430,521570,521990,527340
item_id,,,,,,,,,,,,,,,,,,,,,
10,1.000000,0.982298,0.972275,0.972275,0.929434,0.999960,0.999254,0.999543,0.999960,0.999700,...,0.974205,0.522154,0.972275,0.972275,0.972275,0.972275,0.972275,0.852851,0.972275,0.852851
20,0.982298,1.000000,0.998868,0.998868,0.982102,0.980581,0.988799,0.976187,0.980581,0.986594,...,0.914687,0.672673,0.998868,0.998868,0.998868,0.998868,0.998868,0.739940,0.998868,0.739940
30,0.972275,0.998868,1.000000,1.000000,0.989949,0.970143,0.980581,0.964764,0.970143,0.977715,...,0.894427,0.707107,1.000000,1.000000,1.000000,1.000000,1.000000,0.707107,1.000000,0.707107
40,0.972275,0.998868,1.000000,1.000000,0.989949,0.970143,0.980581,0.964764,0.970143,0.977715,...,0.894427,0.707107,1.000000,1.000000,1.000000,1.000000,1.000000,0.707107,1.000000,0.707107
50,0.929434,0.982102,0.989949,0.989949,1.000000,0.926092,0.942990,0.917857,0.926092,0.938198,...,0.822192,0.800000,0.989949,0.989949,0.989949,0.989949,0.989949,0.600000,0.989949,0.600000


In [ ]:
# convertir el dataframe item_similarity_df a un archivo csv

In [ ]:
# convertir los valores que son iguales a 2 de la columna sentiment_analysis a 1 y los valores 1 a 0
data['sentiment_analysis'] = data['sentiment_analysis'].apply(lambda x: 1 if x == 2 else 0)

In [78]:
data.head(5)

,item_id,recommend,sentiment_analysis,item_name
0,282010,1,0,Carmageddon Max Pack
1,282010,1,0,Carmageddon Max Pack
2,282010,1,0,Carmageddon Max Pack
3,70,1,0,Half-Life
4,70,1,0,Half-Life


In [79]:
df = pd.read_parquet('item_item.parquet')

In [80]:
df.head(5)

,item_id,recommend,sentiment_analysis,item_name
0,282010,1,1,Carmageddon Max Pack
1,282010,1,1,Carmageddon Max Pack
2,282010,1,1,Carmageddon Max Pack
3,70,1,0,Half-Life
4,70,1,0,Half-Life


In [81]:
# Reemplaza los valores 2 por 1 y los valores 1 por 0
df['sentiment_analysis'] = df['sentiment_analysis'].replace({2: 1, 1: 0})

In [82]:
df.head(5)

,item_id,recommend,sentiment_analysis,item_name
0,282010,1,0,Carmageddon Max Pack
1,282010,1,0,Carmageddon Max Pack
2,282010,1,0,Carmageddon Max Pack
3,70,1,0,Half-Life
4,70,1,0,Half-Life


In [83]:
# valores unicos de la column sentiment_analysis
df['sentiment_analysis'].unique()

array([0, 1])

In [ ]:
def user_data(user_id):
    """
    Esta función toma como entrada el ID de un usuario y devuelve un diccionario que contiene información sobre el comportamiento de compra y recomendación del usuario. 
    La información incluye la cantidad total de dinero gastado por el usuario, el porcentaje de juegos que el usuario ha recomendado y la cantidad de artículos únicos que el usuario ha comprado.
    
    Parámetros:
    user_id (str): El ID del usuario.
    
    Devuelve:
    dict_userdata (dict): Un diccionario que contiene información sobre el comportamiento de compra y recomendación del usuario.
    """
    # El resto del código de la función va aquí...

### Nuevo enfoque **ETL**

In [2]:
import pandas as pd


In [11]:
# abrir el archivo parquet
data = pd.read_parquet('games_reviews.parquet')

In [16]:
print(data['item_id'].unique())

[282010     70   2400 ...     30  13230     80]


In [19]:
# mostrar frecuencia de los valores de la columna user_id en un archivo csv
data['user_id'].value_counts().to_csv('user_id.csv')


In [18]:
# frecuencia de los generos
print(data['user_id'].value_counts())

user_id
Cleze                40
snakedog             39
76561198052479552    39
76561198095839864    35
76561198060201321    34
                     ..
stefchygamer          1
Bukky                 1
L_iS_De_Best          1
FullTimeSilver3       1
LaLocaDeLosDulces     1
Name: count, Length: 23157, dtype: Int64
